In [3]:
from tools.web_tool import WebTool
import time

In [4]:
import os
import json
import openai
from pydantic import BaseModel, Field
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Create client
client = openai.OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key='85e577a7bd21434e2d3f1ab2bd7a2750c6db5eb7ddf09cce131655911c93f622',
)

In [5]:
from typing import List

class LoS(BaseModel):
    "Get List of strings to complete the given quries"
    los: List[str] = Field(description="List of five strings to complete the provided task")

class Timeline(BaseModel):
    "Get description about the company and a timeline of major events and achievements in the company's history"
    timeline: List[str] = Field(description="List of major events and achievements happed with the company with detailed description, like founded, acquired, went public and other events.")

class About(BaseModel):
    "Get description products and services offered by the company along with their rough estimated prices"
    description: str = Field(description="Short description about the company.")
    offerings: List[str] = Field(description="List of products or services offered by the company along with their rough prices")
    
class SWOTAnalysis(BaseModel):
    "Market share, sales and SWOT (Strength, weakness, opportunity and threat) analysis of given company"
    #numbers : List[str] = Field(description="List of description of key performance indices like market-share, sales, global precense, etc.")
    strength: List[str] = Field(description="List of strenghts the given company have that make them unique, with detailed description")
    weakness: List[str] = Field(description="List of weakness the given company with detailed description")
    opportunities: List[str] = Field(description="List of opportunities the given company have that will contribute in the company future growth with detailed description")
    threats: List[str] = Field(description="List of threats the given company have that can make business go bust, with detailed description")

In [22]:
def get_message(task,**kwargs):
    if task == 'web-query':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your own intelligence to complete the task."},
            {"role": "user", "content": f"Web search queries for DuckDuckGo search engine for '{kwargs['query']}'"}
        ]
    elif task == 'competitors-name':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your own intelligence and the provided context(if needed) to complete the task. Answer in provided JSON schema."},
            {"role": "user", "content": f"Name mojor competitors in the {kwargs['industry']} business.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    elif task == 'swot-analysis':
        message = [
            {"role": "system", "content": "You are a helpful and experienced market analyst. Use your own intelligence and the provided context(if needed) to complete the SWOT analysis for the given company, dont make things up. Dont use Markdown and Answer in provided JSON schema"},
            {"role": "user", "content": f"Do SWOT analysis on {kwargs['competitor_name']} as a business.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    elif task == 'timeline':
        message = [
            {"role": "system", "content": "You are a helpful researcher. Use your own intelligence and the provided context(if needed) to find the timeline of events about the given company, dont make things up. Dont use Markdown or other formating and Answer in provided json schema"},
            {"role": "user", "content": f"Tell events and achievement happened with {kwargs['competitor_name']} as a business in correct order.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    elif task == 'about':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your own intelligence and the provided context(if needed) to find the description about a company and products and services provided by company, dont make things up. Dont use Markdown or other formating and Answer in provided json schema"},
            {"role": "user", "content": f"Tell description about {kwargs['competitor_name']} as a company and few products or services offer by {kwargs['competitor_name']} along with their pricing.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    else:
        message = [None,None]
    return message

In [23]:
def get_response_helper(response_schema,message,repetetion_penalty):
    print(message)
    response = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=message,
        response_format={"type": "json_object", "schema": response_schema.model_json_schema()},
        frequency_penalty = repetetion_penalty
    )
    print(response.choices[0].message.content)
    content = json.loads(response.choices[0].message.content)
    return content

def get_response(competitor,industry,task,query,schema):
    content = remove_stopwords(WebTool().fetch_content(texts=[query]))[:4096*4]
    clean_content = client.chat.completions.create(
        model='meta-llama/Llama-3-8b-chat-hf',
        messages=[
            {"role": "system", "content": "You are an information retriever and summarizer,ignore everything you know, extract all the factual information regarding the QUERY into a maximum of 300-400 words. Output should be plain text no markdown or formatting just chunks of paragraph full of information, ignore links, using the RAW TEXT"},
            {"role": "user", "content": f"QUERY :\n\n{query}\n\n RAW TEXT :\n\n{content}"} 
        ],
        frequency_penalty = 0.1)
    
    msg = get_message(task=task,
                      industry=industry,
                      competitor_name=competitor,
                      context=clean_content.choices[0].message.content)
    
    report = get_response_helper(response_schema=schema,message=msg,repetetion_penalty=0.23)

    return report

In [24]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_text)
def main(industry):
    #get competitors name
    competitors_name_query = f"Names of top competitors in the {industry} industry."
    competitors_names = get_response(schema=LoS,task='competitors-name',query=competitors_name_query,industry=industry,competitor="")['los']
    final_report = {}
    for competitor in competitors_names[:2]:
    
        queries = [f'SWOT Strength, weakness, opportunity and threat of {competitor} in {industry} industry.',
                   f'Events and achievements of {competitor} in {industry} industry, like investments, sales records, returns, etc.',
                   f'About the {competitor} and the products or services offer by {competitor} in {industry}.']

        swot_report = get_response(industry=industry,competitor=competitor,task='swot-analysis',query=queries[0],schema=SWOTAnalysis)
        final_report[competitor] = swot_report

        about_report = get_response(industry=industry,competitor=competitor,task='timeline',query=queries[1],schema=Timeline)
        final_report[competitor].update(about_report)

        products = get_response(industry=industry,competitor=competitor,task='about',query=queries[2],schema=About)
        final_report[competitor].update(products)

    return final_report

In [25]:
r = main('Laptop')

/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'tech.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.rtings.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'role': 'system', 'content': 'You are a helpful assistant. Use your own intelligence and the provided context(if needed) to complete the task. Answer in provided JSON schema.'}, {'role': 'user', 'content': 'Name mojor competitors in the Laptop business.\n\nCONTEXT :\nHere is the factual information regarding the top competitors in the Laptop industry:\n\nApple is a well-known brand in the laptop industry, offering a range of products including the MacBook Air and MacBook Pro. The company is known for its innovative designs, powerful processors, and high-quality displays.\n\nDell is another major player in the laptop industry, offering a wide range of products including the Inspiron, XPS, and G Series. The company is known for its affordable prices and wide selection of options.\n\nLenovo is a Chinese brand that has gained popularity in recent years, offering a range of products including the ThinkPad and Yoga lines. The company is known for its innovative designs and affordable price

/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'seekingalpha.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'businessmodelanalyst.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'role': 'system', 'content': 'You are a helpful and experienced market analyst. Use your own intelligence and the provided context(if needed) to complete the SWOT analysis for the given company, dont make things up. Dont use Markdown and Answer in provided JSON schema'}, {'role': 'user', 'content': "Do SWOT analysis on Apple as a business.\n\nCONTEXT :\nHere is the factual information regarding the SWOT analysis of Apple in the laptop industry:\n\nApple's Strengths:\n\n* One of the best-known brands in the world\n* Loyal customer base due to innovative products, design quality, and user experience\n* Brand recognition extends to the App Store, making it easy for new users to find apps they need\n* Seen a sheer number of downloads on the App Store, with many developers reporting a huge increase in sales when releasing an app on the platform\n* Known for pushing boundaries and creating innovative products and services that often revolutionize industries\n* Has a leading technological e

/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.infoclutch.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.usatoday.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

[{'role': 'system', 'content': 'You are a helpful and experienced market analyst. Use your own intelligence and the provided context(if needed) to complete the SWOT analysis for the given company, dont make things up. Dont use Markdown and Answer in provided JSON schema'}, {'role': 'user', 'content': 'Do SWOT analysis on Dell as a business.\n\nCONTEXT :\nHere is the factual information regarding the query "SWOT Strength, weakness, opportunity and threat of Dell in Laptop industry":\n\nDell is a multinational computer technology corporation that was founded in February 1984 by Michael Dell. The company is headquartered in Round Rock, Texas, USA and is listed on the New York Stock Exchange under the ticker symbol DELL.\n\nStrengths:\n\n* Excellent Brand Reputation: Ranked #137 in the top regarded companies and #90 in the world\'s most valuable brands.\n* Global Presence: With over 363 offices spread across the world, making it the third-largest PC manufacturer and vendor in the world.\n*

/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dell.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dell.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'role': 'system', 'content': 'You are a helpful researcher. Use your own intelligence and the provided context(if needed) to find the timeline of events about the given company, dont make things up. Dont use Markdown or other formating and Answer in provided json schema'}, {'role': 'user', 'content': 'Tell events and achievement happened with Dell as a business in correct order.\n\nCONTEXT :\nHere is the factual information regarding Dell\'s events and achievements in the laptop industry:\n\nDell has made significant investments in the laptop industry, including a $100 million investment in its manufacturing facility in China in 2019. The company has also expanded its presence in the Indian market, with a new manufacturing facility in Sri City, Andhra Pradesh, which was inaugurated in 2016.\n\nIn terms of sales records, Dell has reported significant growth in its laptop sales in recent years. In 2020, the company reported a 12% year-over-year growth in its laptop sales, with revenue 

/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dell.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dell.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'role': 'system', 'content': 'You are a helpful assistant. Use your own intelligence and the provided context(if needed) to find the description about a company and products and services provided by company, dont make things up. Dont use Markdown or other formating and Answer in provided json schema'}, {'role': 'user', 'content': 'Tell description about Dell as a company and few products or services offer by Dell along with their pricing.\n\nCONTEXT :\nHere is the factual information regarding Dell and its products or services in laptops:\n\nDell offers a range of laptops designed to cater to diverse needs, including business, gaming, and entertainment. Their laptops are equipped with Intel and AMD processors, and often feature cutting-edge technology, high-resolution displays, and robust build quality.\n\nSome of the key features of Dell laptops include:\n\n* Fast charging and ExpressCharge\n* Optimized eye comfort with ComfortView\n* Thin and light designs\n* High-performance optio

In [26]:
import pandas
pandas.DataFrame(r)

,Apple,Dell
strength,"[One of the best-known brands in the world, Lo...","[Excellent Brand Reputation, Global Presence, ..."
weakness,"[High-quality products come at a high cost, ma...","[Overdependence on Tech Sector, Lack of Divers..."
opportunities,"[Has a large and active user base, with over 1...","[Focus on Emerging Economies, Expand Growing D..."
threats,[Faces competition from other technology compa...,"[Shrinking PC Market, Second Wave Pandemic, Lo..."
timeline,"[1976: Apple was founded by Steve Jobs, Steve ...",[2016: Dell inaugurated a new manufacturing fa...
description,Apple is a multinational technology company th...,Dell is a leading technology company that offe...
offerings,"[iPhone, iPad, Mac computers, Apple Watch, App...","[Latitude series: designed for business use, I..."


In [27]:
vars = {
    'competitors':r.keys(),
    'report':r
}

In [28]:
from jinja2 import Environment, FileSystemLoader
env= Environment(loader=FileSystemLoader('templates/'))
temp = env.get_template('competitor.html')
from IPython.core.display import display, HTML
display(HTML(temp.render(vars)))

In [29]:
env= Environment(loader=FileSystemLoader('templates/'))

In [30]:
temp = env.get_template('competitor.html')

In [31]:
from IPython.core.display import display, HTML
display(HTML(temp.render(vars)))

Strengths,Weaknesses
"One of the best-known brands in the world Loyal customer base due to innovative products, design quality, and user experience Brand recognition extends to the App Store, making it easy for new users to find apps they need Seen a sheer number of downloads on the App Store, with many developers reporting a huge increase in sales when releasing an app on the platform Known for pushing boundaries and creating innovative products and services that often revolutionize industries Has a leading technological edge, with a commitment to developing custom chipsets and setting new trends in technology development Has a strong financial position, with a market capitalization of over $1 trillion and annual net income exceeding $50 billion Has a solid network of suppliers, partners, and consumers","High-quality products come at a high cost, making them expensive for competitors Recent releases have tended to feature already-existing technology, making it difficult to stand out and garner a new customer base Highly competitive environment often leads to legal battles over intellectual property rights and trademark disputes Relies heavily on its flagship product, the iPhone, which generates two-thirds of its revenue Has a relatively strict policy on customizing hardware and software, which may limit its appeal to some users Is highly dependent on its proprietary operating system and closed ecosystem, which can restrict compatibility with other devices <!-- Add more weakness points as needed -->"
Opportunities,Threats
"Has a large and active user base, with over 1.8 billion active devices expected to be in use by 2023 Has a strong reputation and widespread presence, allowing it to successfully enter new markets such as China, India, and Russia Can leverage its iconic brand logo and name recognition to tap into new revenue streams and expand its fan base Can capitalize on the growing trend of virtual reality (VR) and augmented reality (AR) by launching VR/AR-based products Can continue to invest in research and development to stay ahead of competitors and push the boundaries of innovation <!-- Add more opportunity points as needed -->","Faces competition from other technology companies such as Google, Samsung, and Xiaomi May struggle to maintain its market share as consumers hold onto their phones for longer and demand for new devices decreases May face challenges from counterfeiters who produce cheaper, lower-quality versions of its products May face economic downturns that reduce demand for its products and affect consumer spending power May face government agencies' attempts to access user data stored on its devices, which could compromise user security and trust. <!-- Add more threat points as needed -->"
Strengths,Weaknesses
Excellent Brand Reputation Global Presence Strong Financial Position Diversified Offerings Increased Investment in R&D Highly Innovative Quality Products Personalized Customization,Overdependence on Tech Sector Lack of Diversification Limited Points of Sale Declining Service Delivery <!-- Add more weakness points as needed -->
Opportunities,Threats
Focus on Emerging Economies Expand Growing Divisions Increase Acquisitions Strengthen Networking Services <!-- Add more opportunity points as needed -->,Shrinking PC Market Second Wave Pandemic Looming Recession Effects of Tariffs Legal Issues <!-- Add more threat points as needed -->


In [1]:
import json
with open('example.json','r') as file:
    j = json.load(file)

In [3]:
from jinja2 import Environment, FileSystemLoader

env= Environment(loader=FileSystemLoader('templates/'))
temp = env.get_template('competitor.html')

vars = {
    'competitors':j.keys(),
    'report':j
}

from IPython.core.display import display, HTML
display(HTML(temp.render(vars)))

Strengths,Weaknesses
"Technological expertise in unmanned aircraft systems (UAS) and a strong reputation for delivering innovative and reliable solutions Strong customer relationships across different sectors including defense, agriculture, energy, and transportation Diverse product portfolio including UAS, electric vehicle charging solutions, and contract services providing a diversified revenue stream",Dependence on government contracts which can be unpredictable and subject to changes in government priorities Vulnerability to regulatory changes as the drone industry is heavily regulated Limited international presence limiting growth opportunities <!-- Add more weakness points as needed -->
Opportunities,Threats
"Growing demand for UAS across different sectors including agriculture, energy, and transportation Increasing adoption of electric vehicles presenting an opportunity to expand product portfolio with charging solutions Growing need for surveillance and reconnaissance capabilities presenting an opportunity to offer UAS solutions to government and commercial customers <!-- Add more opportunity points as needed -->",Intense competition in the drone industry with many established players and new entrants vying for market share Economic downturn impacting Aerovironment's business particularly if government contracts are reduced or delayed Regulatory challenges such as restrictions on use of UAS or increased cost of compliance <!-- Add more threat points as needed -->
Strengths,Weaknesses
"Airobotics has a strong brand reputation and is recognized as a leader in the commercial drone industry The company has a wide range of products and services that cater to different industries and applications Airobotics has a strong focus on innovation and R&D, which enables it to stay ahead of the competition",Airobotics is a relatively new company and may not have the same level of experience and expertise as some of its competitors The company may face challenges in terms of scaling its operations and expanding its global presence Airobotics may face competition from other companies that offer similar products and services <!-- Add more weakness points as needed -->
Opportunities,Threats
"The commercial drone industry is growing rapidly, and Airobotics is well-positioned to capitalize on this trend The company can expand its product and service offerings to cater to different industries and applications Airobotics can leverage its expertise in drone technology to develop new and innovative products and services <!-- Add more opportunity points as needed -->","The commercial drone industry is highly competitive, and Airobotics may face challenges in terms of differentiating itself from its competitors The company may face regulatory challenges and restrictions on the use of drones in certain industries or applications Airobotics may face challenges in terms of maintaining its intellectual property and protecting its proprietary technology <!-- Add more threat points as needed -->"
